<a href="https://colab.research.google.com/github/akhiljoymaliackal/Akhil-Joy/blob/master/mainproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow

In [0]:
import tensorflow as tf
from glob import glob
import numpy as np
import cv2
import os

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
input_width=64
input_height=64
local_input_width=32
local_input_height=32
input_channel=3
input_dim=100
continue_training=False
batch_size=64
train_size=400
Tc=100
Td=1
learning_rate=0.001
momentum=0.5
alpha=0.5
margin=5

In [0]:
checkpoints_path='/content/gdrive/My Drive/Project/checkpoint'
graph_path='/content/gdrive/My Drive/Project/graph'
images_path='/content/gdrive/My Drive/Project/image'



In [0]:
def batch_norm(input, name="batch_norm"):
	with tf.variable_scope(name) as scope:
		input = tf.identity(input)
		channels = input.get_shape()[3]

		offset = tf.get_variable("offset", [channels], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
		scale = tf.get_variable("scale", [channels], dtype=tf.float32, initializer=tf.random_normal_initializer(1.0, 0.02))

		mean, variance = tf.nn.moments(input, axes=[0,1,2], keep_dims=False)

		normalized_batch = tf.nn.batch_normalization(input, mean, variance, offset, scale, variance_epsilon=1e-5)

		return normalized_batch

def linear(input, output_size, name="linear"):
	shape = input.get_shape().as_list()

	with tf.variable_scope(name) as scope:
		matrix = tf.get_variable("W", [shape[1], output_size], tf.float32, tf.random_normal_initializer(stddev=0.02))
		bias = tf.get_variable("bias", [output_size], initializer=tf.constant_initializer(0.0))

		return tf.matmul(input, matrix) + bias

def conv2d(input, out_filter, padding, kernel=5, stride=2, name="conv2d"):
	input_shape = input.get_shape().as_list()
	with tf.variable_scope(name) as scope:
		w = tf.get_variable("w", [kernel, kernel, input_shape[-1], out_filter], initializer=tf.random_normal_initializer(stddev=0.02))
		b = tf.get_variable("b", [out_filter], initializer=tf.constant_initializer(0.0))
		conv = tf.nn.conv2d(input, w,strides=[1, stride, stride, 1],padding=padding)

		conv = tf.reshape(tf.nn.bias_add(conv, b), conv.get_shape())

		return conv

def deconv2d(input, out_shape, name="deconv2d"):
	input_shape = input.get_shape().as_list()
	with tf.variable_scope(name) as scope:
		w = tf.get_variable("w", [4, 4, out_shape[-1], input_shape[-1]], initializer=tf.random_normal_initializer(stddev=0.02))
		b = tf.get_variable("b", [out_shape[-1]], initializer=tf.constant_initializer(0.0))
		deconv = tf.nn.conv2d_transpose(input, w,output_shape=out_shape,strides=[1, 2, 2, 1],padding="SAME")
		deconv = tf.reshape(tf.nn.bias_add(deconv, b), deconv.get_shape())

		return deconv
def dilate_conv2d(input, out_shape, rate, name="dilate_conv2d"):
	input_shape = input.get_shape().as_list()
	with tf.variable_scope(name) as scope:
		w = tf.get_variable("w", [3, 3, input_shape[-1], out_shape[-1]])
		b = tf.get_variable("b", [out_shape[-1]], initializer=tf.constant_initializer(0.0))
		dilate_conv = tf.nn.atrous_conv2d(input,w,rate=rate,padding="SAME")
		dilate_conv = tf.reshape(tf.nn.bias_add(dilate_conv, b), dilate_conv.get_shape())
		return dilate_conv

        #return dilate_conv


# In[28]:


#ops.py
def block_patch(input, margin=5):
	shape = input.get_shape().as_list()

	#create patch in random size
	pad_size = tf.random_uniform([2], minval=15, maxval=25, dtype=tf.int32)
	patch = tf.zeros([pad_size[0], pad_size[1], shape[-1]], dtype=tf.float32)

	h_ = tf.random_uniform([1], minval=margin, maxval=shape[0]-pad_size[0]-margin, dtype=tf.int32)[0]
	w_ = tf.random_uniform([1], minval=margin, maxval=shape[1]-pad_size[1]-margin, dtype=tf.int32)[0]

	padding = [[h_, shape[0]-h_-pad_size[0]], [w_, shape[1]-w_-pad_size[1]], [0, 0]]
	padded = tf.pad(patch, padding, "CONSTANT", constant_values=1)

	coord = h_, w_

	res = tf.multiply(input, padded)

	return res, padded, coord, pad_size

def load_train_data():
	paths = '/content/gdrive/My Drive/Project/dataset/*.jpg'
	data_count = len(glob(paths))

	filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once(paths))

	image_reader = tf.WholeFileReader()
	_, image_file = image_reader.read(filename_queue)
	images = tf.image.decode_jpeg(image_file, channels=3)

	images = tf.image.resize_images(images ,[input_height, input_width])
	images = tf.image.convert_image_dtype(images, dtype=tf.float32) / 127.5 - 1

	orig_images = images
	images, mask, coord, pad_size = block_patch(images, margin=margin)
	mask = tf.reshape(mask, [input_height, input_height, 3])

	mask = -(mask - 1)
	images += mask

	orig_imgs, perturbed_imgs, mask, coord, pad_size = tf.train.shuffle_batch([orig_images, images, mask, coord, pad_size],batch_size=batch_size,capacity=batch_size*2,min_after_dequeue=batch_size)
	return orig_imgs, perturbed_imgs, mask, coord, pad_size, data_count


# In[29]:


class network():
    def __init__(self):

        self.batch_size = batch_size
        self.input_dim = input_dim 

        self.local_width, self.local_height = local_input_width, local_input_height

        self.m = margin

        self.alpha = alpha

        #prepare training data
        self.real_img, self.perturbed_img, self.mask, self.coord, self.pads, self.data_count = load_train_data()
        # self.orig_img, self.test_img, self.test_mask, self.test_data_count = load_test_data(args)
        
        self.single_orig = tf.placeholder(tf.float32, (batch_size, input_height, input_width, 3))
        self.single_test = tf.placeholder(tf.float32, (batch_size, input_height, input_width, 3))
        self.single_mask = tf.placeholder(tf.float32, (batch_size, input_height, input_width, 3))

        self.build_model()
        self.build_loss()

        #summary
        self.recon_loss_sum = tf.summary.scalar("recon_loss", self.recon_loss) 
        self.d_loss_sum = tf.summary.scalar("d_loss", self.d_loss) 
        self.loss_all_sum = tf.summary.scalar("loss_all", self.loss_all)
        self.input_img_sum = tf.summary.image("input_img", self.perturbed_img, max_outputs=5)
        self.real_img_sum = tf.summary.image("real_img", self.real_img, max_outputs=5)
        
        self.recon_img_sum = tf.summary.image("recon_img", self.recon_img, max_outputs=5)
        self.g_local_imgs_sum = tf.summary.image("g_local_imgs", self.g_local_imgs, max_outputs=5)
        self.r_local_imgs_sum = tf.summary.image("r_local_imgs", self.r_local_imgs, max_outputs=5)

    #structure of the model
    def build_model(self):
        def rand_crop(img, coord, pads):
          cropped = tf.image.resize_images(tf.image.crop_to_bounding_box(img, coord[0]-self.m, coord[1]-self.m, pads[0]+self.m*2, pads[1]+self.m*2), (self.local_height, self.local_width))
          return cropped

        # uncomment to concatenate mask and masked input image
        # self.perturbed_img = tf.concat([self.perturbed_img, self.mask], -1)

        self.recon_img, self.g_nets = self.completion_net(self.perturbed_img, name="completion_net")
        self.recon_img = (1-self.mask)*self.real_img + self.mask*self.recon_img

        self.test_res_imgs, _ = self.completion_net(self.single_test, name="completion_net", reuse=True)
        self.test_res_imgs = (1-self.single_mask)*self.single_orig + self.single_mask*self.test_res_imgs

        self.r_local_imgs = []
        self.g_local_imgs = [] 
        for idx in range(0,self.real_img.shape[0]):
            r_cropped = rand_crop(self.real_img[idx], self.coord[idx], self.pads[idx])
            g_cropped = rand_crop(self.recon_img[idx], self.coord[idx], self.pads[idx])
            self.r_local_imgs.append(r_cropped)
            self.g_local_imgs.append(g_cropped)


        self.r_local_imgs = tf.convert_to_tensor(self.r_local_imgs)
        self.g_local_imgs = tf.convert_to_tensor(self.g_local_imgs)
        
        #global discriminator setting
        self.local_fake_d_logits, self.local_fake_d_net = self.local_discriminator(self.g_local_imgs, name="local_discriminator")
        self.local_real_d_logits, self.local_real_d_net = self.local_discriminator(self.r_local_imgs, name="local_discriminator", reuse=True)

        #local discriminator setting
        self.global_fake_d_logits, self.global_fake_d_net = self.global_discriminator(self.recon_img, name="global_discriminator")
        self.global_real_d_logits, self.global_real_d_net = self.global_discriminator(self.real_img, name="global_discriminator", reuse=True)

        self.fake_d_logits = tf.concat([self.local_fake_d_logits, self.global_fake_d_logits], axis=1)
        self.real_d_logits = tf.concat([self.local_real_d_logits, self.global_real_d_logits], axis=1)

        self.fake_loss = linear(self.fake_d_logits, 1, "fake_loss")
        self.real_loss = linear(self.real_d_logits, 1, "real_loss")

        trainable_vars = tf.trainable_variables()
        self.c_vars = []
        self.d_vars = []
        for var in trainable_vars:
            if "completion_net" in var.name:
                self.c_vars.append(var)
            else:
                self.d_vars.append(var)

    #loss function
    def build_loss(self):
        def calc_loss(logits, label):
            if label==1:
                y = tf.ones_like(logits)
            else:
                y = tf.zeros_like(logits)
            return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))

        self.fake_d_loss = calc_loss(self.fake_loss, 0)
        self.real_d_loss = calc_loss(self.real_loss, 1)

        #loss to train the discriminator
        self.d_loss = self.alpha*(self.fake_d_loss + self.real_d_loss)

        self.g_loss = calc_loss(self.fake_loss, 1)
        
        #mse loss in the paper
        self.recon_loss = tf.reduce_mean(tf.nn.l2_loss(self.real_img - self.recon_img))
        
        self.loss_all = self.recon_loss + self.alpha*self.g_loss

    # completion network 
    def completion_net(self, input, name="generator", reuse=False):
        input_shape = input.get_shape().as_list()
        nets = []
        with tf.variable_scope(name, reuse=reuse) as scope:
            conv1 = conv2d(input, 64,kernel=5,stride=1,padding="SAME",name="conv1")
            conv1 = batch_norm(conv1, name="conv_bn1")
            conv1 = tf.nn.relu(conv1)
            
            conv2 = conv2d(conv1, 128,kernel=3,stride=2,padding="SAME",name="conv2")
            conv2 = batch_norm(conv2, name="conv_bn2")
            conv2 = tf.nn.relu(conv2)

            conv3 = conv2d(conv2, 128,kernel=3,stride=1,padding="SAME",name="conv3")
            conv3 = batch_norm(conv3, name="conv_bn3")
            conv3 = tf.nn.relu(conv3)

            conv4 = conv2d(conv3, 256,kernel=3,stride=2,padding="SAME",name="conv4")
            conv4 = batch_norm(conv4, name="conv_bn4")
            conv4 = tf.nn.relu(conv4)

            conv5 = conv2d(conv4, 256,kernel=3,stride=1,padding="SAME",name="conv5")
            conv5 = batch_norm(conv5, name="conv_bn5")
            conv5 = tf.nn.relu(conv5)

            conv6 = conv2d(conv5, 256,kernel=3,stride=1,padding="SAME",name="conv6")
            conv6 = batch_norm(conv5, name="conv_bn6")
            conv6 = tf.nn.relu(conv5)

            #Dilated conv from here
            dilate_conv1 = dilate_conv2d(conv6,[self.batch_size, conv6.get_shape()[1], conv6.get_shape()[2], 256],rate=2,name="dilate_conv1")

            dilate_conv2 = dilate_conv2d(dilate_conv1,[self.batch_size, dilate_conv1.get_shape()[1], dilate_conv1.get_shape()[2], 256],rate=4,name="dilate_conv2")

            dilate_conv3 = dilate_conv2d(dilate_conv2,[self.batch_size, dilate_conv2.get_shape()[1], dilate_conv2.get_shape()[2], 256],rate=8,name="dilate_conv3")

            dilate_conv4 = dilate_conv2d(dilate_conv3,[self.batch_size, dilate_conv3.get_shape()[1], dilate_conv3.get_shape()[2], 256],rate=16,name="dilate_conv4")                                                                                              

            #resize back
            conv7 = conv2d(dilate_conv4, 256,kernel=3,stride=1,padding="SAME",name="conv7")
            conv7 = batch_norm(conv7, name="conv_bn7")
            conv7 = tf.nn.relu(conv7)

            conv8 = conv2d(conv7, 256,kernel=3,stride=1,padding="SAME",name="conv8")
            conv8 = batch_norm(conv8, name="conv_bn8")
            conv8 = tf.nn.relu(conv8)

            deconv1 = deconv2d(conv8, [self.batch_size, input_shape[1]/2, input_shape[2]/2, 128], name="deconv1")
            deconv1 = batch_norm(deconv1, name="deconv_bn1")
            deconv1 = tf.nn.relu(deconv1)

            conv9 = conv2d(deconv1, 128,kernel=3,stride=1,padding="SAME",name="conv9")
            conv9 = batch_norm(conv9, name="conv_bn9")
            conv9 = tf.nn.relu(conv9)

            deconv2 = deconv2d(conv9, [self.batch_size, input_shape[1], input_shape[2], 64], name="deconv2")
            deconv2 = batch_norm(deconv2, name="deconv_bn2")
            deconv2 = tf.nn.relu(deconv2)

            conv10 = conv2d(deconv2, 32,kernel=3,stride=1,padding="SAME",name="conv10")
            conv10 = batch_norm(conv10, name="conv_bn10")
            conv10 = tf.nn.relu(conv10)

            conv11 = conv2d(conv10, 3,kernel=3,stride=1,padding="SAME",name="conv11")
            conv11 = batch_norm(conv11, name="conv_bn11")
            conv11 = tf.nn.tanh(conv11)

            return conv11, nets

    # D network from DCGAN
    def local_discriminator(self, input, name="local_discriminator", reuse=False):
        nets = []
        with tf.variable_scope(name, reuse=reuse) as scope:
            conv1 = tf.contrib.layers.conv2d(input, 64, 5, 2,padding="VALID",activation_fn=None,scope="conv1")
            conv1 = batch_norm(conv1, name="bn1")
            conv1 = tf.nn.relu(conv1)
            nets.append(conv1)

            conv2 = tf.contrib.layers.conv2d(conv1, 128, 5, 2,padding="VALID",activation_fn=None,scope="conv2")
            conv2 = batch_norm(conv2, name="bn2")
            conv2 = tf.nn.relu(conv2)
            nets.append(conv2)

            conv3 = tf.contrib.layers.conv2d(conv2, 256, 5, 2,padding="VALID",activation_fn=None,scope="conv3")
            conv3 = batch_norm(conv1, name="bn3")
            conv3 = tf.nn.relu(conv3)
            nets.append(conv3)

            conv4 = tf.contrib.layers.conv2d(conv3, 512, 5, 2,padding="VALID",activation_fn=None,scope="conv4")
            conv4 = batch_norm(conv4, name="bn4")                                                                                                                           
            conv4 = tf.nn.relu(conv4)
            nets.append(conv4)

            conv5 = tf.contrib.layers.conv2d(conv4, 512, 5, 2,padding="VALID",activation_fn=None,scope="conv5")
            conv5 = batch_norm(conv5, name="bn5")                                                                                                                           
            conv5 = tf.nn.relu(conv5)
            nets.append(conv5)

            flatten = tf.contrib.layers.flatten(conv5)

            output = linear(flatten, 1024, name="linear")

            return output, nets



    def global_discriminator(self, input, name="global_discriminator", reuse=False):
        nets = []
        with tf.variable_scope(name, reuse=reuse) as scope:
            conv1 = tf.contrib.layers.conv2d(input, 64, 5, 2,padding="VALID",activation_fn=None,scope="conv1")
            conv1 = batch_norm(conv1, name="bn1")
            conv1 = tf.nn.relu(conv1)
            nets.append(conv1)

            conv2 = tf.contrib.layers.conv2d(conv1, 128, 5, 2,padding="VALID",activation_fn=None,scope="conv2")
            conv2 = batch_norm(conv2, name="bn2")
            conv2 = tf.nn.relu(conv2)
            nets.append(conv2)

            conv3 = tf.contrib.layers.conv2d(conv2, 256, 5, 2,padding="VALID",activation_fn=None,scope="conv3")
            conv3 = batch_norm(conv1, name="bn3")
            conv3 = tf.nn.relu(conv3)
            nets.append(conv3)

            conv4 = tf.contrib.layers.conv2d(conv3, 512, 5, 2,padding="VALID",activation_fn=None,scope="conv4")
            conv4 = batch_norm(conv4, name="bn4")                                                                                                                           
            conv4 = tf.nn.relu(conv4)
            nets.append(conv4)

            conv5 = tf.contrib.layers.conv2d(conv4, 512, 5, 2,padding="VALID",activation_fn=None,scope="conv5")
            conv5 = batch_norm(conv5, name="bn5")                                                                                                                           
            conv5 = tf.nn.relu(conv5)
            nets.append(conv5)

            conv6 = tf.contrib.layers.conv2d(conv5, 512, 5, 2,padding="VALID",activation_fn=None,scope="conv6")
            conv6 = batch_norm(conv6, name="bn6")                                                                                                                           
            conv6 = tf.nn.relu(conv6)
            nets.append(conv6)


            flatten = tf.contrib.layers.flatten(conv6)

            output = linear(flatten, 1024, name="linear")


            return output, nets


# In[30]:


#train
def train(sess, model):
    #Adam optimizers are used instead of AdaDelta
    d_optimizer = tf.train.AdamOptimizer(learning_rate, beta1=momentum, name="AdamOptimizer_D").minimize(model.d_loss, var_list=model.d_vars)
    c_optimizer = tf.train.AdamOptimizer(learning_rate, beta1=momentum, name="AdamOptimizer_C").minimize(model.recon_loss, var_list=model.c_vars)

    global_optimizer = tf.train.AdamOptimizer(learning_rate, beta1=momentum, name="AdamOptimizer_C").minimize(model.loss_all, var_list=model.c_vars)

    epoch = 0
    step = 0
    global_step = 0

    #saver
    saver = tf.train.Saver()
    if continue_training:
        tf.local_variables_initializer().run()
        last_ckpt = tf.train.latest_checkpoint(checkpoints_path)
        saver.restore(sess, last_ckpt)
        ckpt_name = str(last_ckpt)
        print("Loaded model file from " + ckpt_name)
        epoch = int(ckpt_name.split('-')[-1])
    else:
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()



    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    #summary init
    all_summary = tf.summary.merge([model.recon_loss_sum,
                                    model.d_loss_sum,
                                    model.loss_all_sum,
                                    model.input_img_sum,
                                    model.real_img_sum,
                                    model.recon_img_sum,
                                    model.g_local_imgs_sum,
                                    model.r_local_imgs_sum])
    writer = tf.summary.FileWriter(graph_path, sess.graph)

    #training starts here

    #first train completion network
    while epoch <400:
        print("hai")
        #Training Stage 1 (Completion Network)
        if epoch < Tc:
            summary, c_loss, _ = sess.run([all_summary, model.recon_loss, c_optimizer])
            writer.add_summary(summary, global_step)
            print("Epoch [%d] Step [%d] C Loss: [%.4f]" % (epoch, step, c_loss))
        elif epoch < Tc + Td:
            #Training Stage 2 (Discriminator Network)
            summary, d_loss, _ = sess.run([all_summary, model.d_loss, d_optimizer])
            writer.add_summary(summary, global_step)
            print ("Epoch [%d] Step [%d] D Loss: [%.4f]" % (epoch, step, d_loss))
        else:
            #Training Stage 3 (Completion Network)
            summary, g_loss, _ = sess.run([all_summary, model.loss_all, global_optimizer])
            writer.add_summary(summary, global_step)
            print ("Epoch [%d] Step [%d] C Loss: [%.4f]" % (epoch, step, g_loss))


        # Check Test image results every time epoch is finished
        if step*batch_size >= model.data_count:
            saver.save(sess, checkpoints_path + "/model", global_step=epoch)

            #res_img = sess.run(model.test_res_imgs)

            # save test img result
            #img_tile(epoch, args, res_img)
            step = 0
            epoch += 1

        step += 1
        global_step += 1
       
      
    
     
      


    coord.request_stop()
    coord.join(threads)
    sess.close()
    print("Done.")


def main():
    run_config = tf.ConfigProto()
    run_config.gpu_options.allow_growth = True

    #create graph, images, and checkpoints folder if they don't exist
    if not os.path.exists(checkpoints_path):
        os.makedirs(checkpoints_path)
    if not os.path.exists(graph_path):
        os.makedirs(graph_path)
    if not os.path.exists(images_path):
        os.makedirs(images_path)

    with tf.Session(config=run_config) as sess:
        model = network()

        print('Start Training...')
        train(sess, model)

main()


# In[ ]:




